# Advent of Code 2025

In [42]:
import java.io.File
abstract class Problem<T>(private val filename: String) {
    abstract val testResult: T
    fun run() {
        val testOutput = File("testData/$filename").useLines { solve(it) }
        if (testOutput != testResult) {
            throw IllegalStateException("Test output doesn't match expected result \n Actual: $testOutput \n Expected: $testResult")
        }
        val actualOutput = File("data/$filename").useLines { solve(it) }
        println("Test results: Expected=$testResult Actual=$testOutput")
        println("Solution: $actualOutput")
    }
    abstract fun solve(lines: Sequence<String>): T
}

## Day 1: Secret Entrance
### Part 1
You arrive at the secret entrance to the North Pole base ready to start decorating. Unfortunately, the password seems to have been changed, so you can't get in. A document taped to the wall helpfully explains:

"Due to new security protocols, the password is locked in the safe below. Please see the attached document for the new combination."

The safe has a dial with only an arrow on it; around the dial are the numbers 0 through 99 in order. As you turn the dial, it makes a small click noise as it reaches each number.

The attached document (your puzzle input) contains a sequence of rotations, one per line, which tell you how to open the safe. A rotation starts with an L or R which indicates whether the rotation should be to the left (toward lower numbers) or to the right (toward higher numbers). Then, the rotation has a distance value which indicates how many clicks the dial should be rotated in that direction.

So, if the dial were pointing at `11`, a rotation of `R8` would cause the dial to point at `19`. After that, a rotation of `L19` would cause it to point at `0`.

Because the dial is a circle, turning the dial left from `0` one click makes it point at `99`. Similarly, turning the dial right from `99` one click makes it point at `0`.

So, if the dial were pointing at `5`, a rotation of L10 would cause it to point at `95`. After that, a rotation of `R5` could cause it to point at `0`.

The dial starts by pointing at `50`.

You could follow the instructions, but your recent required official North Pole secret entrance security training seminar taught you that the safe is actually a decoy. The actual password is the number of times the dial is left pointing at `0` after any rotation in the sequence.

For example, suppose the attached document contained the following rotations:
```
L68
L30
R48
L5
R60
L55
L1
L99
R14
L82
```
Following these rotations would cause the dial to move as follows:

The dial starts by pointing at `50`.
The dial is rotated `L68` to point at `82`.
The dial is rotated `L30` to point at `52`.
The dial is rotated `R48` to point at `0`.
The dial is rotated `L5` to point at `95`.
The dial is rotated `R60` to point at `55`.
The dial is rotated `L55` to point at `0`.
The dial is rotated `L1` to point at `99`.
The dial is rotated `L99` to point at `0`.
The dial is rotated `R14` to point at `14`.
The dial is rotated `L82` to point at `32`.
Because the dial points at `0` a total of three times during this process, the password in this example is `3`.

Analyze the rotations in your attached document. What's the actual password to open the door?

In [43]:
class DayOnePartOne(override val testResult: Int = 3) : Problem<Int>("day1.txt") {
    override fun solve(lines: Sequence<String>): Int {
        var current = 50
        var zeroCount = 0
        lines.forEach { line ->
            when (line.first()) {
                'R' -> current += line.drop(1).toInt()
                'L' -> current -= line.drop(1).toInt()
            }

            if (current % 100 == 0) {
                zeroCount++
            }
        }
        return zeroCount
    }
}

DayOnePartOne().run()

Test results: Expected=3 Actual=3
Solution: 1139


### Part 2
You're sure that's the right password, but the door won't open. You knock, but nobody answers. You build a snowman while you think.

As you're rolling the snowballs for your snowman, you find another security document that must have fallen into the snow:

"Due to newer security protocols, please use password method 0x434C49434B until further notice."

You remember from the training seminar that "method 0x434C49434B" means you're actually supposed to count the number of times any click causes the dial to point at 0, regardless of whether it happens during a rotation or at the end of one.

Following the same rotations as in the above example, the dial points at zero a few extra times during its rotations:

The dial starts by pointing at `50`.
The dial is rotated `L68` to point at `82`; during this rotation, it points at `0` once.
The dial is rotated `L30` to point at `52`.
The dial is rotated `R48` to point at `0`.
The dial is rotated `L5` to point at `95`.
The dial is rotated `R60` to point at `55`; during this rotation, it points at `0` once.
The dial is rotated `L55` to point at `0`.
The dial is rotated `L1` to point at `99`.
The dial is rotated `L99` to point at `0`.
The dial is rotated `R14` to point at `14`.
The dial is rotated `L82` to point at `32`; during this rotation, it points at `0` once.
In this example, the dial points at `0` three times at the end of a rotation, plus three more times during a rotation. So, in this example, the new password would be `6`.

Be careful: if the dial were pointing at `50`, a single rotation like `R1000` would cause the dial to point at `0` ten times before returning back to `50`!

Using password method 0x434C49434B, what is the password to open the door?

In [44]:
class DayOnePartTwo(override val testResult: Int = 6) : Problem<Int>("day1.txt") {
    override fun solve(lines: Sequence<String>): Int {
        var current = 50
        var zeroCount = 0
        lines.forEach { line ->
            when (line.first()) {
                'R' -> {
                    repeat(line.drop(1).toInt()) {
                        current++
                        if (current == 100) {
                            zeroCount++
                            current = 0
                        }
                    }
                }
                'L' -> {
                    repeat(line.drop(1).toInt()) {
                        current--
                        when (current) {
                            0 -> zeroCount++
                            -1 -> current = 99
                        }
                    }
                }
            }
        }
        return zeroCount
    }
}

DayOnePartTwo().run()

Test results: Expected=6 Actual=6
Solution: 6684


## Day 2: Invalid IDs
### Part 1
You get inside and take the elevator to its only other stop: the gift shop. "Thank you for visiting the North Pole!" gleefully exclaims a nearby sign. You aren't sure who is even allowed to visit the North Pole, but you know you can access the lobby through here, and from there you can access the rest of the North Pole base.

As you make your way through the surprisingly extensive selection, one of the clerks recognizes you and asks for your help.

As it turns out, one of the younger Elves was playing on a gift shop computer and managed to add a whole bunch of invalid product IDs to their gift shop database! Surely, it would be no trouble for you to identify the invalid product IDs for them, right?

They've even checked most of the product ID ranges already; they only have a few product ID ranges (your puzzle input) that you'll need to check. For example:

```
11-22,95-115,998-1012,1188511880-1188511890,222220-222224,
1698522-1698528,446443-446449,38593856-38593862,565653-565659,
824824821-824824827,2121212118-2121212124
```
(The ID ranges are wrapped here for legibility; in your input, they appear on a single long line.)

The ranges are separated by commas (,); each range gives its first ID and last ID separated by a dash (-).

Since the young Elf was just doing silly patterns, you can find the invalid IDs by looking for any ID which is made only of some sequence of digits repeated twice. So, `55` (5 twice), `6464` (64 twice), and `123123` (123 twice) would all be invalid IDs.

None of the numbers have leading zeroes; `0101` isn't an ID at all. (`101` is a valid ID that you would ignore.)

Your job is to find all of the invalid IDs that appear in the given ranges. In the above example:

`11-22` has two invalid IDs, `11` and `22`.
`95-115` has one invalid ID, `99`.
`998-1012` has one invalid ID, `1010`.
`1188511880-1188511890` has one invalid ID, `1188511885`.
`222220-222224` has one invalid ID, `222222`.
`1698522-1698528` contains no invalid IDs.
`446443-446449` has one invalid ID, `446446`.
`38593856-38593862` has one invalid ID, `38593859`.
The rest of the ranges contain no invalid IDs.
Adding up all the invalid IDs in this example produces `1227775554`.

What do you get if you add up all of the invalid IDs?

In [45]:
class DayTwoPartOne(override val testResult: Long = 1227775554L) : Problem<Long>("day2.txt") {
    override fun solve(lines: Sequence<String>): Long {
        var acc = 0L
        lines.first().split(",").map { it.split("-") }.forEach { split ->
            for (i in split[0].toLong()..split[1].toLong()) {
                val str = i.toString()
                if (str.length % 2 == 0 && str.take(str.length / 2) == str.takeLast(str.length / 2)) {
                    acc += i
                }
            }
        }
        return acc
    }
}

DayTwoPartOne().run()

Test results: Expected=1227775554 Actual=1227775554
Solution: 23534117921


### Part 2
The clerk quickly discovers that there are still invalid IDs in the ranges in your list. Maybe the young Elf was doing other silly patterns as well?

Now, an ID is invalid if it is made only of some sequence of digits repeated at least twice. So, `12341234` (`1234` two times), `123123123` (`123` three times), `1212121212` (`12` five times), and `1111111` (`1` seven times) are all invalid IDs.

From the same example as before:

`11-22` still has two invalid IDs, `11` and `22`.
`95-115` now has two invalid IDs, `99` and `111`.
`998-1012` now has two invalid IDs, `999` and `1010`.
`1188511880-1188511890` still has one invalid ID, `1188511885`.
`222220-222224` still has one invalid ID, `222222`.
`1698522-1698528` still contains no invalid IDs.
`446443-446449` still has one invalid ID, `446446`.
`38593856-38593862` still has one invalid ID, `38593859`.
`565653-565659` now has one invalid ID, `565656`.
`824824821-824824827` now has one invalid ID, `824824824`.
`2121212118-2121212124` now has one invalid ID, `121212121`.
Adding up all the invalid IDs in this example produces `4174379265`.

What do you get if you add up all of the invalid IDs using these new rules?

In [46]:
class DayTwoPartTwo(override val testResult: Long = 4174379265) : Problem<Long>("day2.txt") {
    override fun solve(lines: Sequence<String>): Long {
        var acc = 0L
        lines.first().split(",").map { it.split("-") }.forEach { split ->
            for (i in split[0].toLong()..split[1].toLong()) {
                if (i < 10) continue
                val str = i.toString()
                if (str.all { it == str.first() }) {
                    acc += i
                    continue
                }

                for (j in (str.length / 2) downTo 2) {
                    val chunked = str.chunked(j)
                    if (chunked.size > 1 && chunked.all { it == chunked.first() }) {
                        acc += i
                        break
                    }
                }
            }
        }
        return acc
    }
}

DayTwoPartTwo().run()

Test results: Expected=4174379265 Actual=4174379265
Solution: 31755323497


## Day 3

### Part 1
You descend a short staircase, enter the surprisingly vast lobby, and are quickly cleared by the security checkpoint. When you get to the main elevators, however, you discover that each one has a red light above it: they're all offline.

"Sorry about that," an Elf apologizes as she tinkers with a nearby control panel. "Some kind of electrical surge seems to have fried them. I'll try to get them online soon."

You explain your need to get further underground. "Well, you could at least take the escalator down to the printing department, not that you'd get much further than that without the elevators working. That is, you could if the escalator weren't also offline."

"But, don't worry! It's not fried; it just needs power. Maybe you can get it running while I keep working on the elevators."

There are batteries nearby that can supply emergency power to the escalator for just such an occasion. The batteries are each labeled with their joltage rating, a value from `1` to `9`. You make a note of their joltage ratings (your puzzle input). For example:
```
987654321111111
811111111111119
234234234234278
818181911112111
```
The batteries are arranged into banks; each line of digits in your input corresponds to a single bank of batteries. Within each bank, you need to turn on exactly two batteries; the joltage that the bank produces is equal to the number formed by the digits on the batteries you've turned on. For example, if you have a bank like `12345` and you turn on batteries `2` and `4`, the bank would produce `24` jolts. (You cannot rearrange batteries.)

You'll need to find the largest possible joltage each bank can produce. In the above example:

In `987654321111111`, you can make the largest joltage possible, `98`, by turning on the first two batteries.
In `811111111111119`, you can make the largest joltage possible by turning on the batteries labeled `8` and `9`, producing `89` jolts.
In `234234234234278`, you can make `78` by turning on the last two batteries (marked `7` and `8`).
In `818181911112111`, the largest joltage you can produce is `92`.
The total output joltage is the sum of the maximum joltage from each bank, so in this example, the total output joltage is `98` + `89` + `78` + `92` = `357`.

There are many batteries in front of you. Find the maximum joltage possible from each bank; what is the total output joltage?

In [47]:
class DayThreePartOne(override val testResult: Int = 357) : Problem<Int>("day3.txt") {
    override fun solve(lines: Sequence<String>): Int = lines.sumOf { line ->
        val batteries = line.map { it.digitToInt() }
        val first = batteries.dropLast(1).withIndex().maxBy { it.value }
        val second = batteries.drop(first.index + 1).max()
        "${first.value}$second".toInt()
    }
}

DayThreePartOne().run()

Test results: Expected=357 Actual=357
Solution: 16973


### Part 2
The escalator doesn't move. The Elf explains that it probably needs more joltage to overcome the static friction of the system and hits the big red "joltage limit safety override" button. You lose count of the number of times she needs to confirm "yes, I'm sure" and decorate the lobby a bit while you wait.

Now, you need to make the largest joltage by turning on exactly twelve batteries within each bank.

The joltage output for the bank is still the number formed by the digits of the batteries you've turned on; the only difference is that now there will be 12 digits in each bank's joltage output instead of two.

Consider again the example from before:

987654321111111
811111111111119
234234234234278
818181911112111
Now, the joltages are much larger:

In 987654321111111, the largest joltage can be found by turning on everything except some 1s at the end to produce 987654321111.
In the digit sequence 811111111111119, the largest joltage can be found by turning on everything except some 1s, producing 811111111119.
In 234234234234278, the largest joltage can be found by turning on everything except a 2 battery, a 3 battery, and another 2 battery near the start to produce 434234234278.
In 818181911112111, the joltage 888911112111 is produced by turning on everything except some 1s near the front.
The total output joltage is now much larger: 987654321111 + 811111111119 + 434234234278 + 888911112111 = 3121910778619.

What is the new total output joltage?

In [48]:
class DayThreePartTwo(override val testResult: Long = 3121910778619L) : Problem<Long>("day3.txt") {
    override fun solve(lines: Sequence<String>): Long = lines.sumOf { line ->
        val batteries = line.map { it.digitToInt() }
        var acc = ""
        var indexOfLastBatteryUsed = -1
        for (i in 11 downTo 0) {
            val battery = batteries.withIndex().drop(indexOfLastBatteryUsed + 1).dropLast(i).maxBy { it.value }
            indexOfLastBatteryUsed = battery.index
            acc += battery.value
        }
        acc.toLong()
    }
}

DayThreePartTwo().run()

Test results: Expected=3121910778619 Actual=3121910778619
Solution: 168027167146027


## Day 4
### Part 1
You ride the escalator down to the printing department. They're clearly getting ready for Christmas; they have lots of large rolls of paper everywhere, and there's even a massive printer in the corner (to handle the really big print jobs).

Decorating here will be easy: they can make their own decorations. What you really need is a way to get further into the North Pole base while the elevators are offline.

"Actually, maybe we can help with that," one of the Elves replies when you ask for help. "We're pretty sure there's a cafeteria on the other side of the back wall. If we could break through the wall, you'd be able to keep moving. It's too bad all of our forklifts are so busy moving those big rolls of paper around."

If you can optimize the work the forklifts are doing, maybe they would have time to spare to break through the wall.

The rolls of paper (`@`) are arranged on a large grid; the Elves even have a helpful diagram (your puzzle input) indicating where everything is located.

For example:
```
..@@.@@@@.
@@@.@.@.@@
@@@@@.@.@@
@.@@@@..@.
@@.@@@@.@@
.@@@@@@@.@
.@.@.@.@@@
@.@@@.@@@@
.@@@@@@@@.
@.@.@@@.@.
```
The forklifts can only access a roll of paper if there are fewer than four rolls of paper in the eight adjacent positions. If you can figure out which rolls of paper the forklifts can access, they'll spend less time looking and more time breaking down the wall to the cafeteria.

In this example, there are `13` rolls of paper that can be accessed by a forklift (marked with x):
```
..xx.xx@x.
x@@.@.@.@@
@@@@@.x.@@
@.@@@@..@.
x@.@@@@.@x
.@@@@@@@.@
.@.@.@.@@@
x.@@@.@@@@
.@@@@@@@@.
x.x.@@@.x.
```
Consider your complete diagram of the paper roll locations. How many rolls of paper can be accessed by a forklift?

In [49]:
class DayFourPartOne(override val testResult: Int = 13) : Problem<Int>("day4.txt") {
    private val d = listOf(
        -1 to -1,
        -1 to 0,
        -1 to 1,
        0 to -1,
        0 to 1,
        1 to -1,
        1 to 0,
        1 to 1,
    )

    override fun solve(lines: Sequence<String>): Int {
        val lineList = lines.toList()
        val height = lineList.size + 2
        val width = lineList[0].length + 2
        val matrix = Array(height) { row ->
            Array(width) { column ->
                if (row == 0 || row == height - 1 || column == 0 || column == width - 1) {
                    EMPTY
                } else {
                    lineList[row - 1][column - 1]
                }
            }
        }

        var acc = 0
        for (row in 1..(matrix.size - 2)) {
            for (col in 1..(matrix[0].size - 2)) {
                if (matrix[row][col] == ROLL && d.sumOf { (i, j) -> if (matrix[row + i][col + j] == ROLL) 1 else 0 } < 4) {
                    acc++
                }
            }
        }
        return acc
    }

    companion object {
        private const val ROLL = '@'
        private const val EMPTY = '.'
    }
}

DayFourPartOne().run()

Test results: Expected=13 Actual=13
Solution: 1397


### Part 2
Now, the Elves just need help accessing as much of the paper as they can.

Once a roll of paper can be accessed by a forklift, it can be removed. Once a roll of paper is removed, the forklifts might be able to access more rolls of paper, which they might also be able to remove. How many total rolls of paper could the Elves remove if they keep repeating this process?

Starting with the same example as above, here is one way you could remove as many rolls of paper as possible, using highlighted @ to indicate that a roll of paper is about to be removed, and using x to indicate that a roll of paper was just removed:
```
Initial state:
..@@.@@@@.
@@@.@.@.@@
@@@@@.@.@@
@.@@@@..@.
@@.@@@@.@@
.@@@@@@@.@
.@.@.@.@@@
@.@@@.@@@@
.@@@@@@@@.
@.@.@@@.@.

Remove 13 rolls of paper:
..xx.xx@x.
x@@.@.@.@@
@@@@@.x.@@
@.@@@@..@.
x@.@@@@.@x
.@@@@@@@.@
.@.@.@.@@@
x.@@@.@@@@
.@@@@@@@@.
x.x.@@@.x.

Remove 12 rolls of paper:
.......x..
.@@.x.x.@x
x@@@@...@@
x.@@@@..x.
.@.@@@@.x.
.x@@@@@@.x
.x.@.@.@@@
..@@@.@@@@
.x@@@@@@@.
....@@@...

Remove 7 rolls of paper:
..........
.x@.....x.
.@@@@...xx
..@@@@....
.x.@@@@...
..@@@@@@..
...@.@.@@x
..@@@.@@@@
..x@@@@@@.
....@@@...

Remove 5 rolls of paper:
..........
..x.......
.x@@@.....
..@@@@....
...@@@@...
..x@@@@@..
...@.@.@@.
..x@@.@@@x
...@@@@@@.
....@@@...

Remove 2 rolls of paper:
..........
..........
..x@@.....
..@@@@....
...@@@@...
...@@@@@..
...@.@.@@.
...@@.@@@.
...@@@@@x.
....@@@...

Remove 1 roll of paper:
..........
..........
...@@.....
..x@@@....
...@@@@...
...@@@@@..
...@.@.@@.
...@@.@@@.
...@@@@@..
....@@@...

Remove 1 roll of paper:
..........
..........
...x@.....
...@@@....
...@@@@...
...@@@@@..
...@.@.@@.
...@@.@@@.
...@@@@@..
....@@@...

Remove 1 roll of paper:
..........
..........
....x.....
...@@@....
...@@@@...
...@@@@@..
...@.@.@@.
...@@.@@@.
...@@@@@..
....@@@...

Remove 1 roll of paper:
..........
..........
..........
...x@@....
...@@@@...
...@@@@@..
...@.@.@@.
...@@.@@@.
...@@@@@..
....@@@...
```
Stop once no more rolls of paper are accessible by a forklift. In this example, a total of 43 rolls of paper can be removed.

Start with your original diagram. How many rolls of paper in total can be removed by the Elves and their forklifts?

In [50]:
class DayFourPartTwo(override val testResult: Int = 43) : Problem<Int>("day4.txt") {
    private val d = listOf(
        -1 to -1,
        -1 to 0,
        -1 to 1,
        0 to -1,
        0 to 1,
        1 to -1,
        1 to 0,
        1 to 1,
    )

    override fun solve(lines: Sequence<String>): Int {
        val lineList = lines.toList()
        val height = lineList.size + 2
        val width = lineList[0].length + 2
        val matrix = Array(height) { row ->
            Array(width) { column ->
                if (row == 0 || row == height - 1 || column == 0 || column == width - 1) {
                    EMPTY
                } else {
                    lineList[row - 1][column - 1]
                }
            }
        }

        var acc = 0
        var updates = 0
        do {
            updates = 0
            for (row in 1..(matrix.size - 2)) {
                for (col in 1..(matrix[0].size - 2)) {
                    if (matrix[row][col] == ROLL && d.sumOf { (i, j) -> if (matrix[row + i][col + j] == ROLL) 1 else 0 } < 4) {
                        matrix[row][col] = EMPTY
                        updates++
                        acc++
                    }
                }
            }
        } while (updates != 0)
        return acc
    }

    companion object {
        private const val ROLL = '@'
        private const val EMPTY = '.'
    }
}

DayFourPartTwo().run()

Test results: Expected=43 Actual=43
Solution: 8758
